**RISET MEMBUAT MODEL ML (ALGORITMA CNN)**
*28 Maret 2022 | 21.58*

#SANTAITERARAH

Import berbagai library yg diperlukan

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
pd.set_option("display.precision", 8)

Menghubungkan dengan GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Lokasi dataset yg dipakai

In [ ]:
data_root = '/content/drive/MyDrive/SKRIPSI/Dataset/Wayang/train'

**Pre-processing Data**

Create a data generator for training and validation

In [ ]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)
print(TRAINING_DATA_DIR);

datagen_kwargs = dict(rescale=1./255, validation_split=.20)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(TRAINING_DATA_DIR,subset="validation",shuffle=True,target_size=IMAGE_SHAPE)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(TRAINING_DATA_DIR,subset="training",shuffle=True,target_size=IMAGE_SHAPE)

/content/drive/MyDrive/SKRIPSI/Dataset/Wayang/train
Found 44 images belonging to 2 classes.
Found 176 images belonging to 2 classes.


Learn more about data batches

In [ ]:
image_batch_train, label_batch_train = next(iter(train_generator))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

Image batch shape:  (32, 224, 224, 3)
Label batch shape:  (32, 2)
['Antagonis' 'Protagonis']


In [ ]:
model = tf.keras.Sequential([
hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
    output_shape=[1280],
    trainable=False),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
    ])

model.build([None, 224, 224, 3])
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['acc'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


Finally, we can train our model. You can modify epoches and test the accuracy.

In [ ]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)
hist = model.fit(
    train_generator,
    epochs=10,
    verbose=1,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=val_steps_per_epoch).history

Epoch 1/10
6/6 [==============================] - 69s 11s/step - loss: 0.8442 - acc: 0.5739 - val_loss: 0.5082 - val_acc: 0.7045
Epoch 2/10
6/6 [==============================] - 7s 1s/step - loss: 0.6340 - acc: 0.6818 - val_loss: 0.4354 - val_acc: 0.7955
Epoch 3/10
6/6 [==============================] - 7s 1s/step - loss: 0.5534 - acc: 0.7159 - val_loss: 0.4281 - val_acc: 0.7500
Epoch 4/10
6/6 [==============================] - 7s 1s/step - loss: 0.5170 - acc: 0.7273 - val_loss: 0.4114 - val_acc: 0.8182
Epoch 5/10
6/6 [==============================] - 7s 1s/step - loss: 0.4382 - acc: 0.7614 - val_loss: 0.4017 - val_acc: 0.8182
Epoch 6/10
6/6 [==============================] - 7s 1s/step - loss: 0.3383 - acc: 0.8466 - val_loss: 0.4025 - val_acc: 0.8636
Epoch 7/10
6/6 [==============================] - 7s 1s/step - loss: 0.3996 - acc: 0.8239 - val_loss: 0.4048 - val_acc: 0.8636
Epoch 8/10
6/6 [==============================] - 7s 1s/step - loss: 0.3113 - acc: 0.8636 - val_loss: 0.4146 

Save and Load My Model

In [ ]:
from tensorflow import keras
WAYANG_SAVED_MODEL = '/content/drive/MyDrive/SKRIPSI/Dataset/wayang.h5'
model.save(WAYANG_SAVED_MODEL)
wayang_model = tf.keras.models.load_model((WAYANG_SAVED_MODEL),custom_objects={'KerasLayer':hub.KerasLayer})
 

Image generator

In [ ]:
val_image_batch, val_label_batch = next(iter(valid_generator))
true_label_ids = np.argmax(val_label_batch, axis=-1)
print("Validation batch shape:", val_image_batch.shape)

Validation batch shape: (32, 224, 224, 3)


Predict

In [ ]:
tf_model_predictions = wayang_model.predict(val_image_batch)
tf_pred_dataframe = pd.DataFrame(tf_model_predictions)
tf_pred_dataframe.columns = dataset_labels
print("Prediction results for the first elements")
tf_pred_dataframe.head()

Prediction results for the first elements


,Antagonis,Protagonis
0,0.17338400,0.82661599
1,0.98340458,0.01659545
2,0.00527049,0.99472958
3,0.00475457,0.99524540
4,0.74844545,0.25155455


Show predict

In [ ]:
predicted_ids = np.argmax(tf_model_predictions, axis=-1)
predicted_labels = dataset_labels[predicted_ids]
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(20):
  plt.subplot(6,5,n+1)
  plt.imshow(val_image_batch[n])
  color = "green" if predicted_ids[n] == true_label_ids[n] else "red"
  plt.title(predicted_labels[n].title(), color=color)
  plt.axis('off')
  _ = plt.suptitle("Prediksi Berdasarkan Model (hijau: benar, merah: salah)")

: 